# Import required libraries

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import polars as pl
#import duckdb as dd
#from tqdm import tqdm
import matplotlib.pyplot as plt
#import cv2
#from pydicom import dcmread
import warnings
#from sklearn.preprocessing import LabelEncoder
import pickle
#import gc
import ctypes
import tensorflow as tf
#tf.compat.v1.disable_eager_execution()
#tf.keras.backend.clear_session()

"""for gpu in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(gpu, True)"""
    
import tensorflow_io as tfio
from tensorflow import keras

#tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
try: # detect TPUs
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError: # detect GPUs
    strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
    #strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines

print("Number of accelerators: ", strategy.num_replicas_in_sync)

In [ ]:
# strategy = tf.distribute.MirroredStrategy()
print('DEVICES AVAILABLE: {}'.format(strategy.num_replicas_in_sync))

BATCH_SIZE_PER_REPLICA = 48

#We obtain the BATCH_SIZE dividing by the number of devices. 
#BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * 2

# Feature extraction from image files
## Only if not using tensorflow
### kept here as backward compatibility with earlier work

In [ ]:
metadata_root_path = '/kaggle/input/spinal-canal-stenosis-metadata'

for file in os.listdir(metadata_root_path):
    labels = []
    features = []
    data = {}
    metadata_file_path = os.path.join(metadata_root_path, file)
    print(metadata_file_path)
    metadata_df = pl.read_csv(metadata_file_path, low_memory=True)
    for j in tqdm(range(metadata_df.shape[0])):
        dcm_image_path = metadata_df.item(j,0)
        dicom_ds = dcmread(dcm_image_path)
        img_array = dicom_ds.pixel_array
        features.append(np.mean(img_array.T, axis=0))
        # features.append(dcmread(dcm_image_path).pixel_array)
        labels.append(metadata_df.item(j,2))

    print("feature list length --> ", len(features))
    print("label list length --> ", len(labels))

    #extracted_training_features = np.array([np.resize(img,(128,128)) for img in features])
    #training_labels = np.array(labels)

    #del labels
    #del features
    #gc.collect()

    extracted_training_features_file_name = "{0}_training_features".format(file.replace('_feature_metadata.csv',''))
    labels_file_name = "{0}_labels".format(file.replace('.csv',''))

    with open(extracted_training_features_file_name, "wb") as file:
        pickle.dump(features, file)
    with open(labels_file_name, "wb") as file:
        pickle.dump(labels, file)

    del labels
    del features
    gc.collect()
    libc = ctypes.CDLL("libc.so.6") # clearing cache 
    libc.malloc_trim(0)

    #data["image_array"] = features
    #data["encoded_severity"] = labels
    #extracted_training_data = pd.DataFrame(data)
    #extracted_training_data.to_csv("{0}.csv.gz".format(file.replace('.csv','')), index=False, compression='gzip')

    print('finished dumping features & labels for {0}'.format(file))

# Review pipeline with one sample image

In [ ]:
image_bytes \
= tf.io.read_file('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/100206310/1012284084/1.dcm')

image = tfio.image.decode_dicom_image(image_bytes, scale='auto', dtype=tf.float32)

m, M=tf.math.reduce_min(image), tf.math.reduce_max(image)
image = (tf.image.grayscale_to_rgb(image)-m)/(M-m)
image = tf.image.resize(image, (128,128))

# Preprocessing functions

In [ ]:
def read_and_parse_dicom_files(full_file_path):
    tf.config.run_functions_eagerly(True)
    raw_image = tf.io.read_file(full_file_path)
    sp = tf.strings.split(tf.gather(tf.strings.split(full_file_path, 'images/'), 1), '/')
    N = tf.size(sp)
    LEN = tf.strings.length(tf.gather(sp, 0))+tf.strings.length(tf.gather(sp, 2))
    
    # Add missing file metadata to avoid warnnigs flooding
    if   LEN==12: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x92\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==13: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x92\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==14: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x94\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==15: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x94\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==16: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x96\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==17: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x96\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==18: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x98\x00\x00\x00\x02\x00\x01\x00')
    
    #image_bytes = tf.io.read_file(full_file_path)
    #image = tfio.image.decode_dicom_image(image_bytes, scale='auto', dtype=tf.float32)
    image = tfio.image.decode_dicom_image(raw_image, scale='auto', dtype=tf.float32)
    m, M=tf.math.reduce_min(image), tf.math.reduce_max(image)
    image = (tf.image.grayscale_to_rgb(image)-m)/(M-m)
    image = tf.image.resize(image, (128,128))
    return tf.squeeze(image)

In [ ]:
def load_dataset(image_path, labels):
    image = read_and_parse_dicom_files(image_path)
    return {"images": tf.cast(image, tf.float32), "labels": tf.cast(labels, tf.float32)}

def dict_to_tuple(inputs):
    return inputs["images"], inputs["labels"]

# Train, Test, Validation & holdout splits
## holdout set to be used for CV

In [2]:
from sklearn.model_selection import train_test_split

metadata_file_path = '/kaggle/input/spinal-canal-stenosis-metadata/spinal_canal_stenosis_l1_l2_feature_metadata.csv'
metadata_df = pl.read_csv(metadata_file_path, low_memory=True)

In [3]:
for_train, holdout = train_test_split(metadata_df, test_size=0.4, random_state=42)

x_train, x_test_val = train_test_split(for_train, test_size=0.3, random_state=42)
x_test, x_valid = train_test_split(x_test_val, test_size=0.2, random_state=42)

print("Training data shape : {0}".format(x_train.shape))
print("Test data shape : {0}".format(x_test.shape))
print("Validation data shape : {0}".format(x_valid.shape))
print("Holdout data shape : {0}".format(holdout.shape))

Training data shape : (61794, 3)
Test data shape : (21187, 3)
Validation data shape : (5297, 3)
Holdout data shape : (58853, 3)


In [ ]:
holdout_image_filenames = pl.Series(holdout.select(pl.col('full_img_path'))).to_list()
holdout_image_labels = pl.Series(holdout.select(pl.col('encoded_severity'))).to_list()

train_image_filenames = pl.Series(x_train.select(pl.col('full_img_path'))).to_list()
train_image_labels = pl.Series(x_train.select(pl.col('encoded_severity'))).to_list()

test_image_filenames = pl.Series(x_test.select(pl.col('full_img_path'))).to_list()
test_image_labels = pl.Series(x_test.select(pl.col('encoded_severity'))).to_list()

valid_image_filenames = pl.Series(x_valid.select(pl.col('full_img_path'))).to_list()
valid_image_labels = pl.Series(x_valid.select(pl.col('encoded_severity'))).to_list()

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_image_filenames, train_image_labels))

test_dataset = tf.data.Dataset.from_tensor_slices((test_image_filenames, test_image_labels))

valid_dataset = tf.data.Dataset.from_tensor_slices((valid_image_filenames, valid_image_labels))

holdout_dataset = tf.data.Dataset.from_tensor_slices((holdout_image_filenames, holdout_image_labels))

In [ ]:
train_ds = train_dataset.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.batch(batch_size=BATCH_SIZE, drop_remainder=True)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

test_ds = test_dataset.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(batch_size=BATCH_SIZE, drop_remainder=True)
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)

valid_ds = valid_dataset.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
valid_ds = valid_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
valid_ds = valid_ds.batch(batch_size=BATCH_SIZE, drop_remainder=True)
valid_ds = valid_ds.prefetch(tf.data.AUTOTUNE)

holdout_ds = holdout_dataset.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
holdout_ds = holdout_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
holdout_ds = holdout_ds.batch(batch_size=BATCH_SIZE, drop_remainder=True)
holdout_ds = holdout_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
"""elem = next(iter(train_ds))
elem"""

#### Create a solution dataframe with predictions as 1 or 0 depending on the labels
#### using this solution dataframe and the actual predictions and sample weights (given in the problem statement)
#### build a custom keras metric

In [10]:
pl.Config(fmt_str_lengths=1000)
x_train.head(10)

full_img_path,severity,encoded_severity
str,str,i64
"""/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/581655354/3280289726/17.dcm""","""Normal/Mild""",1
"""/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/1666601651/108284302/27.dcm""","""Normal/Mild""",1
"""/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/2944629826/1953111568/13.dcm""","""Normal/Mild""",1
"""/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/296314829/1382866353/5.dcm""","""Normal/Mild""",1
"""/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/3039901962/171545159/15.dcm""","""Normal/Mild""",1
"""/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/594735110/408469134/21.dcm""","""Normal/Mild""",1
"""/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/1995123254/2154684591/15.dcm""","""Normal/Mild""",1
"""/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/2238966046/2610869166/9.dcm""","""Normal/Mild""",1
"""/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/801319011/1184504151/12.dcm""","""Normal/Mild""",1


In [19]:
def get_study_id(full_image_path):
    return full_image_path.split('/')[-3]

x_train_studies = x_train.with_columns(
    pl.col("full_img_path")
    .map_elements(get_study_id, return_dtype=pl.String)
    .alias("study_id")
)
x_train_studies.head(10)

full_img_path,severity,encoded_severity,study_id
str,str,i64,str
"""/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/581655354/3280289726/17.dcm""","""Normal/Mild""",1,"""581655354"""
"""/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/1666601651/108284302/27.dcm""","""Normal/Mild""",1,"""1666601651"""
"""/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/2944629826/1953111568/13.dcm""","""Normal/Mild""",1,"""2944629826"""
"""/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/296314829/1382866353/5.dcm""","""Normal/Mild""",1,"""296314829"""
"""/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/3039901962/171545159/15.dcm""","""Normal/Mild""",1,"""3039901962"""
"""/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/594735110/408469134/21.dcm""","""Normal/Mild""",1,"""594735110"""
"""/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/1995123254/2154684591/15.dcm""","""Normal/Mild""",1,"""1995123254"""
"""/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/2238966046/2610869166/9.dcm""","""Normal/Mild""",1,"""2238966046"""
"""/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/801319011/1184504151/12.dcm""","""Normal/Mild""",1,"""801319011"""


In [24]:
x_train.select([pl.col('severity'), pl.col('encoded_severity')]).unique()

severity,encoded_severity
str,i64
"""Severe""",2
"""Normal/Mild""",1
"""Moderate""",0


In [9]:
train_studies_metadata_file_path = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train.csv'
train_studies_metadata_df = pl.read_csv(train_studies_metadata_file_path, low_memory=True)
train_studies_metadata_df.select([pl.col('study_id'),pl.col('spinal_canal_stenosis_l1_l2')]).head(10)

study_id,spinal_canal_stenosis_l1_l2
i64,str
4003253,"""Normal/Mild"""
4646740,"""Normal/Mild"""
7143189,"""Normal/Mild"""
8785691,"""Normal/Mild"""
10728036,"""Normal/Mild"""
11340341,"""Normal/Mild"""
11943292,"""Normal/Mild"""
13317052,"""Normal/Mild"""
22191399,"""Normal/Mild"""


In [28]:
def label_encoder(label):
    if label == 'Normal/Mild':
        return 1
    elif label == 'Severe':
        return 2
    else:
        return 0

train_studies_metadata_df_up = train_studies_metadata_df.unpivot(index="study_id")
train_studies_metadata_df_up.columns = ['study_id', 'condition', 'severity']

train_studies_metadata_df_up = train_studies_metadata_df_up.with_columns(
    pl.col("severity")
    .map_elements(label_encoder, return_dtype=pl.Int32)
    .alias("encoded_severity")
)

train_studies_metadata_df_up = train_studies_metadata_df_up.with_columns(
    (pl.col("study_id").cast(pl.String)+'_'+pl.col("condition"))
    .alias("row_id")
)

print(train_studies_metadata_df_up.shape)
train_studies_metadata_df_up.head(10)

(49375, 5)


study_id,condition,severity,encoded_severity,row_id
i64,str,str,i32,str
4003253,"""spinal_canal_stenosis_l1_l2""","""Normal/Mild""",1,"""4003253_spinal_canal_stenosis_l1_l2"""
4646740,"""spinal_canal_stenosis_l1_l2""","""Normal/Mild""",1,"""4646740_spinal_canal_stenosis_l1_l2"""
7143189,"""spinal_canal_stenosis_l1_l2""","""Normal/Mild""",1,"""7143189_spinal_canal_stenosis_l1_l2"""
8785691,"""spinal_canal_stenosis_l1_l2""","""Normal/Mild""",1,"""8785691_spinal_canal_stenosis_l1_l2"""
10728036,"""spinal_canal_stenosis_l1_l2""","""Normal/Mild""",1,"""10728036_spinal_canal_stenosis_l1_l2"""
11340341,"""spinal_canal_stenosis_l1_l2""","""Normal/Mild""",1,"""11340341_spinal_canal_stenosis_l1_l2"""
11943292,"""spinal_canal_stenosis_l1_l2""","""Normal/Mild""",1,"""11943292_spinal_canal_stenosis_l1_l2"""
13317052,"""spinal_canal_stenosis_l1_l2""","""Normal/Mild""",1,"""13317052_spinal_canal_stenosis_l1_l2"""
22191399,"""spinal_canal_stenosis_l1_l2""","""Normal/Mild""",1,"""22191399_spinal_canal_stenosis_l1_l2"""


In [33]:
temp = train_studies_metadata_df_up\
.select([pl.col('row_id'), pl.col('encoded_severity'), pl.col('severity')])

In [38]:
temp.filter(pl.col('encoded_severity').is_null())

row_id,encoded_severity,severity
str,i32,str
"""3008676218_spinal_canal_stenosis_l1_l2""",null,null
"""3008676218_spinal_canal_stenosis_l2_l3""",null,null
"""3008676218_spinal_canal_stenosis_l3_l4""",null,null
"""3008676218_spinal_canal_stenosis_l4_l5""",null,null
"""3008676218_spinal_canal_stenosis_l5_s1""",null,null
…,…,…
"""1671676307_right_subarticular_stenosis_l5_s1""",null,null
"""2048265504_right_subarticular_stenosis_l5_s1""",null,null
"""3024532039_right_subarticular_stenosis_l5_s1""",null,null


In [34]:
train_studies_metadata_df_final = temp\
.select([pl.col('row_id'), pl.col('encoded_severity'), pl.col('severity')])\
.pivot("severity", index="row_id", values="encoded_severity")

train_studies_metadata_df_final.columns = ['row_id', 'normal_mild', 'moderate', 'severe', 'dispose']

train_studies_metadata_df_final.head(10)

row_id,normal_mild,moderate,severe,dispose
str,i32,i32,i32,i32
"""4003253_spinal_canal_stenosis_l1_l2""",1,null,null,null
"""4646740_spinal_canal_stenosis_l1_l2""",1,null,null,null
"""7143189_spinal_canal_stenosis_l1_l2""",1,null,null,null
"""8785691_spinal_canal_stenosis_l1_l2""",1,null,null,null
"""10728036_spinal_canal_stenosis_l1_l2""",1,null,null,null
"""11340341_spinal_canal_stenosis_l1_l2""",1,null,null,null
"""11943292_spinal_canal_stenosis_l1_l2""",1,null,null,null
"""13317052_spinal_canal_stenosis_l1_l2""",1,null,null,null
"""22191399_spinal_canal_stenosis_l1_l2""",1,null,null,null


In [5]:
config = {}
config['root_file_path'] = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images'
config['start'] = 10
config['end'] = 110
studies = os.listdir(config['root_file_path'])[config['start']:config['end']]

test_dict = {}
for study in studies:
    image_files = []
    for dirname, _, filenames in os.walk(config['root_file_path']+'/'+study):
        for filename in filenames:
            test_dict[os.path.join(dirname, filename).split('/')[-3]] = image_files
            image_files.append(os.path.join(dirname, filename))

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers

with strategy.scope():
    
    rsna_input = layers.Input(shape=(128,128,3), name="rsna_input")
    
    conv_base = EfficientNetB0(include_top=False, weights="imagenet", input_tensor=rsna_input)
    conv_base.trainable = False
    
    x = layers.GlobalAveragePooling2D(name="avg_pool")(conv_base.output)
    x = layers.BatchNormalization()(x)
    
    hidden_layer1 = layers.Dense(200, activation="relu", kernel_initializer=keras.initializers.LecunNormal(seed=None))(x)
    hidden_layer2 = layers.Dense(100, activation="selu")(hidden_layer1)
    hidden_layer3 = layers.Dense(50, activation="selu")(hidden_layer2)
    rsna_output = layers.Dense(3, activation="softmax")(hidden_layer3)
    #model = tf.keras.models.Model(inputs=rsna_input, outputs=rsna_output)
    model = tf.keras.Model(rsna_input, rsna_output)
    
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("keras_effnet_spinal_canal_stenosis_l1_l2.keras")
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

    model.compile(loss="sparse_categorical_crossentropy", optimizer="adamax", metrics=["accuracy"])

In [ ]:
config = model.get_config()
print(config["layers"][0]["config"])

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_ds, epochs=5, validation_data=valid_ds, callbacks=[checkpoint_cb, early_stopping_cb])

In [ ]:
pd.DataFrame(history.history).plot(figsize=(10,6))
plt.grid(True)
plt.gca().set_ylim(0.60,1)
plt.show()

In [ ]:
model.save("keras_base_spinal_canal_stenosis_l1_l2.h5")

In [ ]:
cropped_image = tf.image.resize_with_crop_or_pad(image, 100, 100)

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(5,5))
axes.imshow(np.squeeze(cropped_image.numpy()), cmap='gray')
axes.set_title('image')

In [ ]:
model = keras.models.\
load_model("/kaggle/input/keras_base_scs_l1_l2/tensorflow2/default/1/keras_base_spinal_canal_stenosis_l1_l2.h5")

### Remove the squeeze from image file preprocessing

In [ ]:
def read_and_parse_dicom_files_for_inf(full_file_path):
    tf.config.run_functions_eagerly(True)
    raw_image = tf.io.read_file(full_file_path)
    sp = tf.strings.split(tf.gather(tf.strings.split(full_file_path, 'images/'), 1), '/')
    N = tf.size(sp)
    LEN = tf.strings.length(tf.gather(sp, 0))+tf.strings.length(tf.gather(sp, 2))
    
    # Add missing file metadata to avoid warnnigs flooding
    if   LEN==12: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x92\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==13: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x92\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==14: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x94\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==15: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x94\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==16: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x96\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==17: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x96\x00\x00\x00\x02\x00\x01\x00')
    elif LEN==18: raw_image = tf.strings.regex_replace(raw_image, pattern=b'DICM\x02\x00\x01\x00', rewrite=b'DICM\x02\x00\x00\x00UL\x04\x00\x98\x00\x00\x00\x02\x00\x01\x00')
    
    #image_bytes = tf.io.read_file(full_file_path)
    #image = tfio.image.decode_dicom_image(image_bytes, scale='auto', dtype=tf.float32)
    image = tfio.image.decode_dicom_image(raw_image, scale='auto', dtype=tf.float32)
    m, M=tf.math.reduce_min(image), tf.math.reduce_max(image)
    image = (tf.image.grayscale_to_rgb(image)-m)/(M-m)
    image = tf.image.resize(image, (128,128))
    return image

In [ ]:
test_image = \
read_and_parse_dicom_files_for_inf('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3481971518/1.dcm')

In [ ]:
test_image.shape

In [ ]:
y_proba = model.predict(test_image)

In [ ]:
y_proba[0]

In [ ]:
np.array([0.333333, 0.333333, 0.333333])

In [ ]:
model.evaluate(test_ds)

In [ ]:
severity_levels = ['normal_mild', 'moderate', 'severe']
rows = {'44036939_spinal_canal_stenosis_l1_l2': y_proba[0]}

In [ ]:
for key, value in rows.items():
    value_reshaped = value.reshape(1, -1)
    predictions = np.ascontiguousarray(value_reshaped)
    df = pd.DataFrame(predictions, columns=severity_levels)
    df.insert(loc=0, column='row_id', value=key)

In [ ]:
df